In [7]:
import numpy as np
import pandas as pd

**Gradient boosting for MAE** <br>
let $f0 = median(y)$ <br>
For $m =1$ to $M$
   1. Compute pseudo residuals <br>
    $r = sign(y - f_m(x))$
   2. Fit a regression tree $T_m$ to r.  Here dataset is $(x_i, r_i)$
   3. Recompute the predictions of the regression tree using the **median** of $y_i-f_m(x_i)$
   4. $f_{m + 1} = f_m + T_m$ <br>
Return $f_M$

In [48]:
x = np.array([750, 800, 850, 900, 950])
y = np.array([1160, 1200, 1280, 1450, 2000])

In [49]:
np.median(y)

1280.0

In [50]:
res = y - np.median(y)
res

array([-120.,  -80.,    0.,  170.,  720.])

In [16]:
def sum_of_errors(y):
    mu = y.mean()
    return ((y - mu)**2).sum()

In [51]:
f0 = np.median(y)*np.ones(5)
res1 = y - f0
r1 = np.sign(y - f0 + 1e-10) # small epsilon to avoid getting a 0

In [52]:
df = pd.DataFrame({"x":x, "y":y, "f0":f0, "res1": res1, "r1":r1})
df

,x,y,f0,res1,r1
0,750,1160,1280.0,-120.0,-1.0
1,800,1200,1280.0,-80.0,-1.0
2,850,1280,1280.0,0.0,1.0
3,900,1450,1280.0,170.0,1.0
4,950,2000,1280.0,720.0,1.0


The split on r1 is trivial on x between 800 and 850
What are the values of T1?
We have to take the median on the terminal nodes so we get:

In [53]:
res1[:2]

array([-120.,  -80.])

In [54]:
m1 = np.median(res1[:2])
m2 = np.median(res1[2:])
m1, m2

(-100.0, 170.0)

In [56]:
T1 = np.concatenate([m1*np.ones(2), m2*np.ones(3)])
T1

array([-100., -100.,  170.,  170.,  170.])

In [57]:
df["T1"] = T1
df

,x,y,f0,res1,r1,T1
0,750,1160,1280.0,-120.0,-1.0,-100.0
1,800,1200,1280.0,-80.0,-1.0,-100.0
2,850,1280,1280.0,0.0,1.0,170.0
3,900,1450,1280.0,170.0,1.0,170.0
4,950,2000,1280.0,720.0,1.0,170.0


In [58]:
df["f1"] = f0 + T1
df

,x,y,f0,res1,r1,T1,f1
0,750,1160,1280.0,-120.0,-1.0,-100.0,1180.0
1,800,1200,1280.0,-80.0,-1.0,-100.0,1180.0
2,850,1280,1280.0,0.0,1.0,170.0,1450.0
3,900,1450,1280.0,170.0,1.0,170.0,1450.0
4,950,2000,1280.0,720.0,1.0,170.0,1450.0


In [60]:
res2 = y - df["f1"].values
res2

array([ -20.,   20., -170.,    0.,  550.])

In [61]:
r2 = np.sign(res2 + 1e-10)
r2

array([-1.,  1., -1.,  1.,  1.])

In [62]:
df["r2"] = r2
df

,x,y,f0,res1,r1,T1,f1,r2
0,750,1160,1280.0,-120.0,-1.0,-100.0,1180.0,-1.0
1,800,1200,1280.0,-80.0,-1.0,-100.0,1180.0,1.0
2,850,1280,1280.0,0.0,1.0,170.0,1450.0,-1.0
3,900,1450,1280.0,170.0,1.0,170.0,1450.0,1.0
4,950,2000,1280.0,720.0,1.0,170.0,1450.0,1.0
